In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib

from geolib import geohash

import geopandas as gpd
import matplotlib.pyplot as plt
#from shapely.geometry import Point
from shapely import Point,Polygon
import geopy
import geopy.distance
#import geohash2
from geodatasets import get_path,data
from ipywidgets import interact,Select,Dropdown
import folium
from tqdm import tqdm
from IPython.display import display

import sys
sys.path.append('../')

from utils import (ProcessData,
                   trans_chara,
                   fill_missing_values,
                   fill_missing_with_mode,
                   find_nearest_station_main_table,
                   calculate_distance,
                   convert_kanji_to_fullwidth_2,
                   encode_geohash,
                   to_cat_col,
                   TreeExecution,
                   VotingModel,
                   submisson_datetime,
                   check_dataframe_elements
                   )

TOTAL_PRICE_LOG="取引価格（総額）_log"
PRICE_PER_AREA_LOG="単位面積あたりの取引価格_log"

PROJECT_DIRECYORY=r"C:\Users\yuto2\OneDrive\ドキュメント\Pythonフォルダ\Nishika\中古マンション価格予測_2024夏の部\project_directory"

DATA_INTERMEDIATE_RESULTS_CHECKPOINTS=r"C:\Users\yuto2\OneDrive\ドキュメント\Pythonフォルダ\Nishika\中古マンション価格予測_2024夏の部\project_directory\data\intermediate_results\checkpoints"

#TARGET="取引価格（総額）_log"

In [2]:
df=pd.read_csv(os.path.join(DATA_INTERMEDIATE_RESULTS_CHECKPOINTS,"preprocess_main_table.csv"),index_col=0)
df=df[['dataset', 'ID', '価格情報区分', '都道府県名', '市区町村名', '地区名', '住所不一致_Flag', '住所',
       '最寄駅：名称', '最寄駅：距離（分）', '徒歩時間_10分単位', 'x_api', 'y_api', 'lon', 'lat',
       '間取り', '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等',"取引価格（総額）_log", '単位面積あたりの取引価格_log', '建築西暦年', '築年数',
       'log_面積（㎡）', '購入までの築年数', '面積（㎡）-最寄駅：距離（分）', '面積（㎡）-築年数',
       '面積（㎡）容積率（％）_combi', '旧耐震フラグ', '最寄駅までの距離', 'geohash_2', 'geohash_3',
       'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7', '県_市_count',
       '建築年_count', '県_市_geohash_2_駅数', '県_市_geohash_3_駅数', '県_市_geohash_4_駅数',
       '県_市_geohash_5_駅数', '県_市_geohash_6_駅数', '県_市_geohash_7_駅数',
       '県_市_geohash_2_駅徒歩時間平均', '県_市_geohash_2_建築年平均', '県_市_geohash_3_駅徒歩時間平均',
       '県_市_geohash_3_建築年平均', '県_市_geohash_4_駅徒歩時間平均', '県_市_geohash_4_建築年平均',
       '県_市_geohash_5_駅徒歩時間平均', '県_市_geohash_5_建築年平均', '県_市_geohash_6_駅徒歩時間平均',
       '県_市_geohash_6_建築年平均', '県_市_geohash_7_駅徒歩時間平均', '県_市_geohash_7_建築年平均']]


In [3]:
check_dataframe_elements(df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1063673 entries, 0 to 1063672
Data columns (total 64 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   dataset                1063673 non-null  object 
 1   ID                     1063673 non-null  int64  
 2   価格情報区分                 1063673 non-null  object 
 3   都道府県名                  1063673 non-null  object 
 4   市区町村名                  1063673 non-null  object 
 5   地区名                    1063608 non-null  object 
 6   住所不一致_Flag             1063673 non-null  int64  
 7   住所                     1063673 non-null  object 
 8   最寄駅：名称                 1063616 non-null  object 
 9   最寄駅：距離（分）              1014618 non-null  float64
 10  徒歩時間_10分単位             1014618 non-null  float64
 11  x_api                  1063616 non-null  float64
 12  y_api                  1063616 non-null  float64
 13  lon                    1051220 non-null  float64
 14  lat               

In [4]:
# データフレームを分割
df_train = df[df['dataset'] == 'train'].set_index('ID')
df_test = df[df['dataset'] == 'test'].set_index('ID')

# datasetカラムを削除
df_train = df_train.drop(columns=['dataset'])
df_test = df_test.drop(columns=['dataset'])

df_train,cat_cols=to_cat_col(df_train)

# インデックスを読み込み
train_index = np.load(os.path.join(DATA_INTERMEDIATE_RESULTS_CHECKPOINTS,"train_index_l1_cv.npy"))
test_index = np.load(os.path.join(DATA_INTERMEDIATE_RESULTS_CHECKPOINTS,'test_index_l1_cv.npy'))
print(train_index)
print(test_index)
# インデックスを設定して順序を再現
df_train = df_train.loc[train_index]
df_test = df_test.loc[test_index]

print("Train DataFrame:")
print(df_train)
print("\nTest DataFrame:")
print(df_test)

[ 1046933  1079053  1170588 ... 47003811 47010902 47011639]
[ 1000107  1000109  1000112 ... 47037850 47037855 47037856]
Train DataFrame:
             価格情報区分 都道府県名   市区町村名    地区名  住所不一致_Flag              住所 最寄駅：名称  \
ID                                                                            
1046933   不動産取引価格情報   北海道  札幌市白石区    南郷通           0    北海道札幌市白石区南郷通  南郷７丁目   
1079053   不動産取引価格情報   北海道   札幌市西区  発寒１２条           0   北海道札幌市西区発寒１２条   発寒中央   
1170588      成約価格情報   北海道     江別市   大麻北町           0      北海道江別市大麻北町   森林公園   
1001637      成約価格情報   北海道  札幌市中央区   北５条西           0   北海道札幌市中央区北５条西  西２８丁目   
1014790   不動産取引価格情報   北海道  札幌市中央区  宮の森２条           0  北海道札幌市中央区宮の森２条  西２８丁目   
...             ...   ...     ...    ...         ...             ...    ...   
47001427  不動産取引価格情報   沖縄県     那覇市  首里山川町           0     沖縄県那覇市首里山川町     儀保   
47035975     成約価格情報   沖縄県  中頭郡北谷町     美浜           0     沖縄県中頭郡北谷町美浜  てだこ浦西   
47003811  不動産取引価格情報   沖縄県     那覇市    古波蔵           0       沖縄県那覇市古波蔵     

In [5]:
%%script false --no-raise-error

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from ipywidgets import interact, Select, Dropdown,HBox
from IPython.display import display
import matplotlib.pyplot as plt
import base64
from io import BytesIO
#import folium

# ジオハッシュの境界ボックスを計算する関数を定義
def geohash_to_polygon(gh):
    if pd.isna(gh):
        return None
    bounds = geohash.bounds(gh)
    return Polygon([
        (bounds[0][1], bounds[0][0]),
        (bounds[0][1], bounds[1][0]),
        (bounds[1][1], bounds[1][0]),
        (bounds[1][1], bounds[0][0]),
        (bounds[0][1], bounds[0][0])
    ])

# データフレームを読み込み、必要な列をグループ化してジオハッシュの境界ボックスを計算
df = df_train.groupby(["取引時点", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
# カウント集約を追加
df_count = df_train.groupby(["取引時点", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"]).size().reset_index(name='count')

# 各ジオハッシュレベルでのデータを個別にフィルタリング
def process_geohash_level(level):
    df_mean = df.groupby(["取引時点", level])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
    df_mean['geometry'] = [geohash_to_polygon(gh) for gh in tqdm(df_mean[level],desc=f"Calculating geohash polygons:{level}")]

    df_count_level = df_count.groupby(["取引時点", level])['count'].sum().reset_index()
    df_merged = pd.merge(df_mean, df_count_level, on=["取引時点", level])

    gdf = gpd.GeoDataFrame(df_merged, geometry='geometry')
    gdf.set_crs(epsg=4326, inplace=True)
    
    return gdf

gdf_geohash_2 = process_geohash_level("geohash_2")
gdf_geohash_3 = process_geohash_level("geohash_3")
gdf_geohash_4 = process_geohash_level("geohash_4")
gdf_geohash_5 = process_geohash_level("geohash_5")
gdf_geohash_6 = process_geohash_level("geohash_6")
gdf_geohash_7 = process_geohash_level("geohash_7")

# 全てのジオハッシュレベルのデータフレームを連結
gdf_all = pd.concat([gdf_geohash_2, gdf_geohash_3, gdf_geohash_4, gdf_geohash_5, gdf_geohash_6, gdf_geohash_7])
unique_dates = gdf_all['取引時点'].unique()
unique_geohash_levels = ['geohash_2', 'geohash_3', 'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7']
date_selector = Select(options=unique_dates, description='取引時点:')
geohash_selector = Dropdown(options=unique_geohash_levels, description='Geohash Level:')

selectors=HBox([date_selector, geohash_selector])
# 横並びに配置
def create_selectors():
    return HBox([date_selector, geohash_selector])
#display(selectors)


def plot_price_trend(selected_geohash):
    df_selected = df[df[selected_geohash].notnull()]
    trend_data = df_selected.groupby("取引時点")[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
    
    fig, ax = plt.subplots()
    ax.plot(trend_data["取引時点"], trend_data[f"{TOTAL_PRICE_LOG}"])
    ax.set_title(f'{selected_geohash}の価格推移')
    ax.set_xlabel('取引時点')
    ax.set_ylabel('平均価格')
    
    # 画像をバイトストリームに変換
    tmpfile = BytesIO()
    fig.savefig(tmpfile, format='png')
    encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')
    
    html = '<img src="data:image/png;base64,{}">'.format(encoded)
    iframe = folium.IFrame(html, width=500, height=250)
    popup = folium.Popup(iframe, max_width=500)
    
    return popup

def update_map(selected_date, selected_geohash):
    m = folium.Map(location=[35.0, 135.0], zoom_start=10)
    gdf_dict = {
        'geohash_2': gdf_geohash_2,
        'geohash_3': gdf_geohash_3,
        'geohash_4': gdf_geohash_4,
        'geohash_5': gdf_geohash_5,
        'geohash_6': gdf_geohash_6,
        'geohash_7': gdf_geohash_7
    }
    selected_gdf = gdf_dict[selected_geohash][gdf_dict[selected_geohash]['取引時点'] == selected_date]

    # Choropleth layer for the average price
    folium.Choropleth(
        geo_data=selected_gdf,
        name='Average Price',
        data=selected_gdf,
        columns=[selected_geohash, f"{TOTAL_PRICE_LOG}"],
        key_on=f'feature.properties.{selected_geohash}',
        fill_color='YlGnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='平均価格',
        bins=[3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0]  # 固定のビンを設定
    ).add_to(m)

    # Add a feature group for the counts
    folium.GeoJson(
        selected_gdf,
        name='Counts',
        style_function=lambda x: {
            'fillColor': '#ffaf00',
            'color': '#ffaf00',
            'weight': 1,
            'dashArray': '5, 5'
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['count', f"{TOTAL_PRICE_LOG}"],
            aliases=['Count:', 'Average Price:'],
            localize=True
        )
    ).add_to(m)

    # 価格推移グラフをプロットとして追加
    popup = plot_price_trend(selected_geohash)
    folium.Marker([35.0, 135.0], popup=popup).add_to(m)

    folium.LayerControl().add_to(m)

    display(m)

# インタラクティブに地図を更新する
interact(update_map, selected_date=date_selector, selected_geohash=geohash_selector)

Couldn't find program: 'false'


In [6]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from ipywidgets import interact, Select, Dropdown,HBox
import time
import ipywidgets as widgets
from IPython.display import display, clear_output


# ジオハッシュの境界ボックスを計算する関数を定義
def geohash_to_polygon(gh):
    if pd.isna(gh):
        return None
    bounds = geohash.bounds(gh)
    return Polygon([
        (bounds[0][1], bounds[0][0]),
        (bounds[0][1], bounds[1][0]),
        (bounds[1][1], bounds[1][0]),
        (bounds[1][1], bounds[0][0]),
        (bounds[0][1], bounds[0][0])
    ])

# データフレームを読み込み、必要な列をグループ化してジオハッシュの境界ボックスを計算
df = df_train.groupby(["取引時点", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
# カウント集約を追加
df_count = df_train.groupby(["取引時点", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"]).size().reset_index(name='count')

# 各ジオハッシュレベルでのデータを個別にフィルタリング
def process_geohash_level(level):
    df_mean = df.groupby(["取引時点", level])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
    df_mean['geometry'] = [geohash_to_polygon(gh) for gh in tqdm(df_mean[level],desc=f"Calculating geohash polygons:{level}")]

    df_count_level = df_count.groupby(["取引時点", level])['count'].sum().reset_index()
    df_merged = pd.merge(df_mean, df_count_level, on=["取引時点", level])

    gdf = gpd.GeoDataFrame(df_merged, geometry='geometry')
    gdf.set_crs(epsg=4326, inplace=True)
    
    return gdf

gdf_geohash_2 = process_geohash_level("geohash_2")
gdf_geohash_3 = process_geohash_level("geohash_3")
gdf_geohash_4 = process_geohash_level("geohash_4")
gdf_geohash_5 = process_geohash_level("geohash_5")
gdf_geohash_6 = process_geohash_level("geohash_6")
gdf_geohash_7 = process_geohash_level("geohash_7")

# 全てのジオハッシュレベルのデータフレームを連結
gdf_all = pd.concat([gdf_geohash_2, gdf_geohash_3, gdf_geohash_4, gdf_geohash_5, gdf_geohash_6, gdf_geohash_7])
unique_dates = gdf_all['取引時点'].unique()
unique_geohash_levels = ['geohash_2', 'geohash_3', 'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7']
date_selector = Select(options=unique_dates, description='取引時点:')
geohash_selector = Dropdown(options=unique_geohash_levels, description='Geohash Level:')

selectors=HBox([date_selector, geohash_selector])
# 横並びに配置
def create_selectors():
    return HBox([date_selector, geohash_selector])
#display(selectors)

# ボタンを作成
play_button = widgets.Button(description="Play")
output = widgets.Output()

# update_map関数内の処理
def update_map_loop(selected_geohash):
    gdf_dict = {
        'geohash_2': gdf_geohash_2,
        'geohash_3': gdf_geohash_3,
        'geohash_4': gdf_geohash_4,
        'geohash_5': gdf_geohash_5,
        'geohash_6': gdf_geohash_6,
        'geohash_7': gdf_geohash_7
    }
    
    with output:
        for date in unique_dates:
            clear_output(wait=True)
            m = folium.Map(location=[35.0, 135.0], zoom_start=1)
            selected_gdf = gdf_dict[selected_geohash][gdf_dict[selected_geohash]['取引時点'] == date]

            folium.Choropleth(
                geo_data=selected_gdf,
                name='Average Price',
                data=selected_gdf,
                columns=[selected_geohash, f"{TOTAL_PRICE_LOG}"],
                key_on=f'feature.properties.{selected_geohash}',
                fill_color='YlGnBu',
                fill_opacity=0.7,
                line_opacity=0.2,
                legend_name='平均価格',
                bins=[3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0]  # 固定のビンを設定
            ).add_to(m)

            folium.GeoJson(
                selected_gdf,
                name='Counts',
                style_function=lambda x: {
                    'fillColor': '#ffaf00',
                    'color': '#ffaf00',
                    'weight': 1,
                    'dashArray': '5, 5'
                },
                tooltip=folium.GeoJsonTooltip(
                    fields=['count', f"{TOTAL_PRICE_LOG}"],
                    aliases=['Count:', 'Average Price:'],
                    localize=True
                )
            ).add_to(m)

            folium.LayerControl().add_to(m)
            display(m)
            time.sleep(1)  # ループの各ステップ間に1秒の遅延を追加

def on_button_clicked(b):
    selected_geohash = geohash_selector.value
    update_map_loop(selected_geohash)

play_button.on_click(on_button_clicked)

# 表示
display(HBox([date_selector, geohash_selector, play_button]), output)

Calculating geohash polygons:geohash_7: 100%|██████████| 381944/381944 [01:41<00:00, 3753.18it/s]


Output()

In [7]:
print(abc)

NameError: name 'abc' is not defined

In [8]:
#無難で完成度高い

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from ipywidgets import interact, Select, Dropdown,HBox
from IPython.display import display

# ジオハッシュの境界ボックスを計算する関数を定義
def geohash_to_polygon(gh):
    if pd.isna(gh):
        return None
    bounds = geohash.bounds(gh)
    return Polygon([
        (bounds[0][1], bounds[0][0]),
        (bounds[0][1], bounds[1][0]),
        (bounds[1][1], bounds[1][0]),
        (bounds[1][1], bounds[0][0]),
        (bounds[0][1], bounds[0][0])
    ])

# データフレームを読み込み、必要な列をグループ化してジオハッシュの境界ボックスを計算
df = df_train.groupby(["取引時点", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
# カウント集約を追加
df_count = df_train.groupby(["取引時点", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"]).size().reset_index(name='count')

# 各ジオハッシュレベルでのデータを個別にフィルタリング
def process_geohash_level(level):
    df_mean = df.groupby(["取引時点", level])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
    df_mean['geometry'] = [geohash_to_polygon(gh) for gh in tqdm(df_mean[level],desc=f"Calculating geohash polygons:{level}")]

    df_count_level = df_count.groupby(["取引時点", level])['count'].sum().reset_index()
    df_merged = pd.merge(df_mean, df_count_level, on=["取引時点", level])

    gdf = gpd.GeoDataFrame(df_merged, geometry='geometry')
    gdf.set_crs(epsg=4326, inplace=True)
    
    return gdf

gdf_geohash_2 = process_geohash_level("geohash_2")
gdf_geohash_3 = process_geohash_level("geohash_3")
gdf_geohash_4 = process_geohash_level("geohash_4")
gdf_geohash_5 = process_geohash_level("geohash_5")
gdf_geohash_6 = process_geohash_level("geohash_6")
gdf_geohash_7 = process_geohash_level("geohash_7")

# 全てのジオハッシュレベルのデータフレームを連結
gdf_all = pd.concat([gdf_geohash_2, gdf_geohash_3, gdf_geohash_4, gdf_geohash_5, gdf_geohash_6, gdf_geohash_7])
unique_dates = gdf_all['取引時点'].unique()
unique_geohash_levels = ['geohash_2', 'geohash_3', 'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7']
date_selector = Select(options=unique_dates, description='取引時点:')
geohash_selector = Dropdown(options=unique_geohash_levels, description='Geohash Level:')

selectors=HBox([date_selector, geohash_selector])
# 横並びに配置
def create_selectors():
    return HBox([date_selector, geohash_selector])
#display(selectors)


# 地図を更新する関数を定義
def update_map(selected_date, selected_geohash):
    m = folium.Map(location=[35.0, 135.0], zoom_start=5)
    if selected_geohash == 'geohash_2':
        selected_gdf = gdf_geohash_2[gdf_geohash_2['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_3':
        selected_gdf = gdf_geohash_3[gdf_geohash_3['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_4':
        selected_gdf = gdf_geohash_4[gdf_geohash_4['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_5':
        selected_gdf = gdf_geohash_5[gdf_geohash_5['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_6':
        selected_gdf = gdf_geohash_6[gdf_geohash_6['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_7':
        selected_gdf = gdf_geohash_7[gdf_geohash_7['取引時点'] == selected_date]

    # Choropleth layer for the average price
    folium.Choropleth(
        geo_data=selected_gdf,
        name='choropleth',
        data=selected_gdf,
        columns=[selected_geohash, f"{TOTAL_PRICE_LOG}"],
        key_on=f'feature.properties.{selected_geohash}',
        fill_color='YlGnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='平均価格',
        bins=[3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0]  # 固定のビンを設定
    ).add_to(m)

    # Add a feature group for the counts
    folium.GeoJson(
        selected_gdf,
        name='Counts',
        style_function=lambda x: {
            'fillColor': '#ffaf00',
            'color': '#ffaf00',
            'weight': 1,
            'dashArray': '5, 5'
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['count', f"{TOTAL_PRICE_LOG}"],
            aliases=['Count:', 'Average Price:'],
            localize=True
        )
    ).add_to(m)

    folium.LayerControl().add_to(m)

    display(m)

# インタラクティブに地図を更新する
interact(update_map, selected_date=date_selector, selected_geohash=geohash_selector)





Calculating geohash polygons:geohash_7: 100%|██████████| 381944/381944 [00:54<00:00, 6977.77it/s]


interactive(children=(Select(description='取引時点:', options=(2005.75, 2005.99, 2006.25, 2006.5, 2006.75, 2006.99…

<function __main__.update_map(selected_date, selected_geohash)>

In [ ]:
print(abc)

NameError: name 'abc' is not defined

In [ ]:
print(abc)

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from ipywidgets import interact, Select, Dropdown
from IPython.display import display

# ジオハッシュの境界ボックスを計算
def geohash_to_polygon(gh):
    #print(gh)
    if pd.isna(gh):
        return None
    
    bounds = geohash.bounds(gh)
    return Polygon([
        (bounds[0][1], bounds[0][0]),
        (bounds[0][1], bounds[1][0]),
        (bounds[1][1], bounds[1][0]),
        (bounds[1][1], bounds[0][0]),
        (bounds[0][1], bounds[0][0])
    ])

# データフレームを読み込み、必要な列をグループ化してジオハッシュの境界ボックスを計算
df = df_train.groupby(["取引時点", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()

# 各ジオハッシュレベルでのデータを個別にフィルタリング
df_geohash_2 = df.groupby(["取引時点", "geohash_2"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df_geohash_2['geometry'] = [geohash_to_polygon(geohash) for geohash in tqdm(df_geohash_2['geohash_2'],desc=f"Calculating geohash polygons level:2")]
gdf_geohash_2 = gpd.GeoDataFrame(df_geohash_2, geometry='geometry')
print(gdf_geohash_2)
print("-"*50)

df_geohash_3 = df.groupby(["取引時点", "geohash_3"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df_geohash_3['geometry'] = [geohash_to_polygon(geohash) for geohash in tqdm(df_geohash_3['geohash_3'],desc=f"Calculating geohash polygons level:3")]
gdf_geohash_3 = gpd.GeoDataFrame(df_geohash_3, geometry='geometry')
print(gdf_geohash_3)
print("-"*50)

df_geohash_4 = df.groupby(["取引時点", "geohash_4"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df_geohash_4['geometry'] = [geohash_to_polygon(geohash) for geohash in tqdm(df_geohash_4['geohash_4'],desc=f"Calculating geohash polygons level:4")]
gdf_geohash_4 = gpd.GeoDataFrame(df_geohash_4, geometry='geometry')

df_geohash_5 = df.groupby(["取引時点", "geohash_5"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df_geohash_5['geometry'] = [geohash_to_polygon(geohash) for geohash in tqdm(df_geohash_5['geohash_5'],desc=f"Calculating geohash polygons level:5")]
gdf_geohash_5 = gpd.GeoDataFrame(df_geohash_5, geometry='geometry')

df_geohash_6 = df.groupby(["取引時点", "geohash_6"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df_geohash_6['geometry'] = [geohash_to_polygon(geohash) for geohash in tqdm(df_geohash_6['geohash_6'],desc=f"Calculating geohash polygons level:6")]
gdf_geohash_6 = gpd.GeoDataFrame(df_geohash_6, geometry='geometry')

df_geohash_7 = df.groupby(["取引時点", "geohash_7"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df_geohash_7['geometry'] = [geohash_to_polygon(geohash) for geohash in tqdm(df_geohash_7['geohash_7'],desc=f"Calculating geohash polygons level:7")]
gdf_geohash_7 = gpd.GeoDataFrame(df_geohash_7, geometry='geometry')

# 全てのジオハッシュレベルのデータフレームを連結
gdf_all = pd.concat([gdf_geohash_2, gdf_geohash_3, gdf_geohash_4, gdf_geohash_5, gdf_geohash_6, gdf_geohash_7])
#print(gdf_all)
unique_dates = gdf_all['取引時点'].unique()
unique_geohash_levels = ['geohash_2', 'geohash_3', 'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7']
date_selector = Select(options=unique_dates, description='取引時点:')
geohash_selector = Dropdown(options=unique_geohash_levels, description='Geohash Level:')

# 地図を更新する関数を定義
def update_map_(selected_date, selected_geohash):
    m = folium.Map(location=[35.0, 135.0], zoom_start=5)
    if selected_geohash == 'geohash_2':
        selected_gdf = gdf_geohash_2[gdf_geohash_2['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_3':
        selected_gdf = gdf_geohash_3[gdf_geohash_3['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_4':
        selected_gdf = gdf_geohash_4[gdf_geohash_4['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_5':
        selected_gdf = gdf_geohash_5[gdf_geohash_5['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_6':
        selected_gdf = gdf_geohash_6[gdf_geohash_6['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_7':
        selected_gdf = gdf_geohash_7[gdf_geohash_7['取引時点'] == selected_date]

    for _, row in selected_gdf.iterrows():
        if row['geometry'] is not None:
            folium.GeoJson(row['geometry'].__geo_interface__, name=row[selected_geohash]).add_to(m)
    display(m)


In [ ]:

# 地図を更新する関数を定義
def update_map(selected_date, selected_geohash):
    m = folium.Map(location=[35.0, 135.0], zoom_start=5)
    if selected_geohash == 'geohash_2':
        selected_gdf = gdf_geohash_2[gdf_geohash_2['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_3':
        selected_gdf = gdf_geohash_3[gdf_geohash_3['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_4':
        selected_gdf = gdf_geohash_4[gdf_geohash_4['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_5':
        selected_gdf = gdf_geohash_5[gdf_geohash_5['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_6':
        selected_gdf = gdf_geohash_6[gdf_geohash_6['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_7':
        selected_gdf = gdf_geohash_7[gdf_geohash_7['取引時点'] == selected_date]

    folium.Choropleth(
        geo_data=selected_gdf,
        name='choropleth',
        data=selected_gdf,
        columns=[selected_geohash, f"{TOTAL_PRICE_LOG}"],
        key_on=f'feature.properties.{selected_geohash}',
        fill_color='YlGnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='平均価格'
    ).add_to(m)

    folium.LayerControl().add_to(m)

    display(m)

interact(update_map, selected_date=date_selector, selected_geohash=geohash_selector)



In [ ]:
df = df_train.groupby(["取引時点", "geohash_2","geohash_3","geohash_4","geohash_5","geohash_6","geohash_7"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from ipywidgets import interact, Select, Dropdown
from IPython.display import display

# ジオハッシュの境界ボックスを計算する関数を定義
# ジオハッシュの境界ボックスを計算
def geohash_to_polygon(gh):
    #print(gh)
    if pd.isna(gh):
        return None
    
    bounds = geohash.bounds(gh)
    return Polygon([
        (bounds[0][1], bounds[0][0]),
        (bounds[0][1], bounds[1][0]),
        (bounds[1][1], bounds[1][0]),
        (bounds[1][1], bounds[0][0]),
        (bounds[0][1], bounds[0][0])
    ])


# データフレームを読み込み、必要な列をグループ化してジオハッシュの境界ボックスを計算
df = df_train.groupby(["取引時点", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df['geometry_2'] = [geohash_to_polygon(geohash) for geohash in df['geohash_2']]
df['geometry_3'] = [geohash_to_polygon(geohash) for geohash in df['geohash_3']]
df['geometry_4'] = [geohash_to_polygon(geohash) for geohash in df['geohash_4']]
df['geometry_5'] = [geohash_to_polygon(geohash) for geohash in df['geohash_5']]
df['geometry_6'] = [geohash_to_polygon(geohash) for geohash in df['geohash_6']]
df['geometry_7'] = [geohash_to_polygon(geohash) for geohash in df['geohash_7']]
gdf = gpd.GeoDataFrame(df)
print(gdf)
print(abc)
# 取引時点とジオハッシュレベルを選択できるドロップダウンメニューを作成
unique_dates = gdf['取引時点'].unique()
unique_geohash_levels = ['geohash_2', 'geohash_3', 'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7']
date_selector = Select(options=unique_dates, description='取引時点:')
geohash_selector = Dropdown(options=unique_geohash_levels, description='Geohash Level:')

# 地図を更新する関数を定義
def update_map(selected_date, selected_geohash):
    m = folium.Map(location=[35.0, 135.0], zoom_start=5)
    selected_gdf = gdf[gdf['取引時点'] == selected_date]
    for _, row in selected_gdf.iterrows():
        if row[f"geometry_{selected_geohash[-1]}"] is not None:
            folium.GeoJson(row[f'geometry_{selected_geohash[-1]}'].__geo_interface__, name=row[selected_geohash]).add_to(m)
    display(m)

interact(update_map, selected_date=date_selector, selected_geohash=geohash_selector)


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
#from shapely.geometry import Point
from shapely import Point,Polygon
import geopy
import geopy.distance
#import geohash2
from geodatasets import get_path,data
from ipywidgets import interact,Select,Dropdown
import folium
from tqdm import tqdm
from IPython.display import display

# サンプルデータを作成
#data = {
#    'city': ['Tokyo', 'Osaka', 'Nagoya', 'Fukuoka'],
#    'latitude': [35.6895, 34.6937, 35.1815, 33.5902],
#    'longitude': [139.6917, 135.5022, 136.9066, 130.4017],
#}

# ジオハッシュをデータに追加
#data['geohash'] = [geohash2.encode(lat, lon, 5) for lat, lon in zip(data['latitude'], data['longitude'])]
#df = df_train.groupby(["取引時点", "geohash_2"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()


# ジオハッシュの境界ボックスを計算
def geohash_to_polygon(gh):
    #print(gh)
    if pd.isna(gh):
        return None
    
    bounds = geohash.bounds(gh)
    return Polygon([
        (bounds[0][1], bounds[0][0]),
        (bounds[0][1], bounds[1][0]),
        (bounds[1][1], bounds[1][0]),
        (bounds[1][1], bounds[0][0]),
        (bounds[0][1], bounds[0][0])
    ])

# 境界ボックスをデータに追加
for i in range(2,8):
#    df[f'geometry_{i}'] = [geohash_to_polygon(geohash) for geohash in df[f'geohash_{i}']]
    df[f'geometry_{i}'] = [geohash_to_polygon(geohash) for geohash in tqdm(df[f'geohash_{i}'], desc=f"Calculating geohash polygons: Level{i}")]
#df[f'geometry_2'] = [geohash_to_polygon(geohash) for geohash in tqdm(df[f'geohash_2'], desc="Calculating geohash polygons")]

gdf = gpd.GeoDataFrame(df, geometry='geometry_2')
print(gdf.shape)
print(gdf)

unique_datas=gdf["取引時点"].unique()
unique_geohash_levels = ['geohash_2', 'geohash_3', 'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7']
date_selector=Select(options=unique_datas,description="取引時点:")
geohash_selector = Dropdown(options=unique_geohash_levels, description='Geohash Level:')

# 地図を更新する関数を定義
def update_map(selected_date, selected_geohash):
    m = folium.Map(location=[35.0, 135.0], zoom_start=5)
    selected_gdf = gdf[gdf['取引時点'] == selected_date]
    for _, row in selected_gdf.iterrows():
        if row[f"geometry_{selected_geohash[-1]}"] is not None:
            folium.GeoJson(row[f'geometry_{selected_geohash[-1]}'].__geo_interface__, name=row[selected_geohash]).add_to(m)
    display(m)

interact(update_map, selected_date=date_selector, selected_geohash=geohash_selector)


def update_map(selected_date):
    m = folium.Map(location=[35.0, 135.0], zoom_start=5)
    selected_gdf = gdf[gdf['取引時点'] == selected_date]
    for _, row in selected_gdf.iterrows():
        if row["geometry_2"] is not None:
            folium.GeoJson(row['geometry_2'].__geo_interface__, name=row['geohash_2']).add_to(m)
    display(m)

#interact(update_map, selected_date=date_selector)

# Foliumの地図を作成
#m = folium.Map(location=[35.0, 135.0], zoom_start=5)


# ジオハッシュの境界ボックスを地図に追加
#for _, row in tqdm(gdf.iterrows(),total=gdf.shape[0],desc="Adding polygons to map"):
#    if row["geometry_2"] is not None:
#        folium.GeoJson(row['geometry_2'].__geo_interface__, name=row['geohash_2']).add_to(m)

# 地図を保存
#m.save('map.html')

# 地図を表示
#m


In [ ]:
print(abc)

In [ ]:
print(abc)

In [ ]:
print(abc)

In [ ]:
# データフレームを分割
df_train = df[df['dataset'] == 'train'].set_index('ID')
df_test = df[df['dataset'] == 'test'].set_index('ID')

# datasetカラムを削除
df_train = df_train.drop(columns=['dataset'])
df_test = df_test.drop(columns=['dataset'])

df_train,cat_cols=to_cat_col(df_train)

# インデックスを読み込み
train_index = np.load(os.path.join(DATA_INTERMEDIATE_RESULTS_CHECKPOINTS,"train_index_l1_cv.npy"))
test_index = np.load(os.path.join(DATA_INTERMEDIATE_RESULTS_CHECKPOINTS,'test_index_l1_cv.npy'))
print(train_index)
print(test_index)
# インデックスを設定して順序を再現
df_train = df_train.loc[train_index]
df_test = df_test.loc[test_index]

print("Train DataFrame:")
print(df_train)
print("\nTest DataFrame:")
print(df_test)

In [ ]:
average_price_per_year=df_train.groupby("取引時点")[f"{TOTAL_PRICE_LOG}"].mean()
print(average_price_per_year)
# 可視化
plt.figure(figsize=(10, 6))
average_price_per_year.plot(kind='line')
plt.title('年ごとの平均取引価格の推移')
plt.xlabel('年')
plt.ylabel('平均取引価格（総額）_log')
plt.grid(True)
plt.show()

In [ ]:
print(abc)

In [ ]:
tree_class=TreeExecution(df_train,y_name=PRICE_PER_AREA_LOG,cat_cols=cat_cols)
#model_lgbm=tree_class.lightgbm_simple()
fitted_models_lgbm,mae_score,oof_df=tree_class.lightgbm_l1_cv()

In [ ]:
oof_df.to_csv("/content/drive/MyDrive/yuto/project_directory/data/cv_predictions/sub_oof_lgbm_l1_cv.csv")
oof_df

In [ ]:
#model_cat=VotingModel(fitted_models_cat,cat_cols)
model_lgbm = VotingModel(fitted_models_lgbm,cat_cols)
#model_xg=VotingModel(fitted_models_xg,cat_cols)
#y_pred_cat=pd.Series(model_cat.predict_cat(df_test.drop(f"{TARGET}",axis=1)),index=df_test.index)
y_pred_lgbm=pd.Series(model_lgbm.predict_lgbm(df_test.drop(f"{PRICE_PER_AREA_LOG}",axis=1)),index=df_test.index)
#y_pred_xg=pd.Series(model_xg.predict_xg(df_test_tree.drop("取引価格（総額）_log",axis=1)),index=df_test_tree.index)


y_pred=y_pred_lgbm
# 単位面積あたりの取引価格の対数から10のべき乗を取る
price_per_area = 10**y_pred_lgbm
# その値に面積を掛けて、取引価格（総額）を計算
predicted_price_total = price_per_area * df_test["面積（㎡）"]
# 取引価格（総額）の対数を計算
y_pred= np.log10(predicted_price_total)
#y_pred=y_pred.rename(columns={"面積（㎡）":"取引価格（総額）_log"})
# Seriesの名前を変更
y_pred.name = "取引価格（総額）_log"

#y_pred=y_pred_cat
#y_pred=y_pred_lgbm
# y_predをデータフレームに変換
df_pred = pd.DataFrame(y_pred, columns=["取引価格（総額）_log"])
#print(df_pred)
#print("--------------------------------------------------------------------------------")
# インデックスをリセットし、カラム名を'ID'に変更
#df_pred.reset_index(inplace=True)
#print(df_pred)
#print("--------------------------------------------------------------------------------")
#df_pred.rename(columns={'index': 'ID'}, inplace=True)
#print(df_pred)
#print("--------------------------------------------------------------------------------")

# 結果を表示
print(df_pred)

now_str=submisson_datetime()
df_pred.to_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/one_model_predictions/sub_test_lgbm_l1_cv.csv")


In [ ]:
df_sub=pd.read_csv("/content/drive/MyDrive/yuto/project_directory/data/final_predictions/test_lgbm_cv20240908_111634.csv")
print(df_sub)

In [ ]:
def ML_Record(df, model, cv_score, now_str, model_name):
    # 例として、取得したカラム名のリスト
    columns = df.columns.tolist()

    # モデルがアンサンブルか単一のモデルかをチェック
    if hasattr(model, 'estimators'):
        # 各estimatorのパラメータを取得し、全て同じかチェック
        params_list = [estimator.get_params() for estimator in model.estimators]
        # 最初のモデルのパラメータを基準に比較
        base_params = params_list[0]
        all_same = all(params == base_params for params in params_list)

        if all_same:
            print("All estimators have the same parameters.")
        else:
            print("Not all estimators have the same parameters.")

        # 各estimatorのパラメータを表示
        for i, params in enumerate(params_list):
            print(f"Estimator {i} parameters: {params}")
    else:
        # 単一のモデルの場合
        base_params = model.get_params()
        print("Single model parameters:", base_params)

    # ピリオドをアンダースコアに置き換えたファイル名
    cv_score_str = str(cv_score).replace(".", "_")

    # モデルの保存
    joblib.dump(model, f"/content/drive/MyDrive/yuto/project_directory/models/sub_model_{model_name}_{cv_score_str}.pkl")

    # jsonファイル作成
    data_to_save = {
        'columns': columns,
        f'{model_name}_params': base_params,
        'cv_score': cv_score,
        'lb_score': None
    }
    filename = f"/content/drive/MyDrive/yuto/project_directory/data/model_params/sub_model_{model_name}_{cv_score_str}.json"
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)

In [ ]:
ML_Record(df_train, model_lgbm, mae_score, now_str, model_name="lgbm_l1_cv")

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from ipywidgets import interact, Select, Dropdown
from IPython.display import display

def geohash_to_polygon(gh):
    #print(gh)
    if pd.isna(gh):
        return None
    
    bounds = geohash.bounds(gh)
    return Polygon([
        (bounds[0][1], bounds[0][0]),
        (bounds[0][1], bounds[1][0]),
        (bounds[1][1], bounds[1][0]),
        (bounds[1][1], bounds[0][0]),
        (bounds[0][1], bounds[0][0])
    ])

# データフレームを読み込み、必要な列をグループ化してジオハッシュの境界ボックスを計算
df = df_train.groupby(["取引時点", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()

# カウント集約を追加
df_count = df_train.groupby(["取引時点", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"]).size().reset_index(name='count')

# 各ジオハッシュレベルでのデータを個別にフィルタリング
def process_geohash_level(level):
    df_mean = df.groupby(["取引時点", level])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
    df_mean['geometry'] = [geohash_to_polygon(gh) for gh in df_mean[level]]
    
    df_count_level = df_count.groupby(["取引時点", level])['count'].sum().reset_index()
    df_merged = pd.merge(df_mean, df_count_level, on=["取引時点", level])
    
    gdf = gpd.GeoDataFrame(df_merged, geometry='geometry')
    gdf.set_crs(epsg=4326, inplace=True)
    
    return gdf

gdf_geohash_2 = process_geohash_level("geohash_2")
gdf_geohash_3 = process_geohash_level("geohash_3")
print(gdf_geohash_3)
gdf_geohash_4 = process_geohash_level("geohash_4")
gdf_geohash_5 = process_geohash_level("geohash_5")
gdf_geohash_6 = process_geohash_level("geohash_6")
gdf_geohash_7 = process_geohash_level("geohash_7")

# 全てのジオハッシュレベルのデータフレームを連結
gdf_all = pd.concat([gdf_geohash_2, gdf_geohash_3, gdf_geohash_4, gdf_geohash_5, gdf_geohash_6, gdf_geohash_7])
unique_dates = gdf_all['取引時点'].unique()
unique_geohash_levels = ['geohash_2', 'geohash_3', 'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7']
date_selector = Select(options=unique_dates, description='取引時点:')
geohash_selector = Dropdown(options=unique_geohash_levels, description='Geohash Level:')

# 地図を更新する関数を定義
def update_map(selected_date, selected_geohash):
    m = folium.Map(location=[35.0, 135.0], zoom_start=5)
    if selected_geohash == 'geohash_2':
        selected_gdf = gdf_geohash_2[gdf_geohash_2['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_3':
        selected_gdf = gdf_geohash_3[gdf_geohash_3['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_4':
        selected_gdf = gdf_geohash_4[gdf_geohash_4['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_5':
        selected_gdf = gdf_geohash_5[gdf_geohash_5['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_6':
        selected_gdf = gdf_geohash_6[gdf_geohash_6['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_7':
        selected_gdf = gdf_geohash_7[gdf_geohash_7['取引時点'] == selected_date]
    
    # Choropleth layer for the average price
    folium.Choropleth(
        geo_data=selected_gdf,
        name='choropleth',
        data=selected_gdf,
        columns=[selected_geohash, f"{TOTAL_PRICE_LOG}"],
        key_on=f'feature.properties.{selected_geohash}',
        fill_color='YlGnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='平均価格'
    ).add_to(m)

    # Add a feature group for the counts
    counts_column = f"{selected_geohash}_count"
    selected_gdf[counts_column] = selected_gdf.groupby(selected_geohash)[selected_geohash].transform('count')

    folium.GeoJson(
        selected_gdf,
        name='counts',
        style_function=lambda x: {
            'fillColor': '#ffaf00',
            'color': '#ffaf00',
            'weight': 1,
            'dashArray': '5, 5'
        },
        tooltip=folium.GeoJsonTooltip(fields=[counts_column], aliases=['Count:'])
    ).add_to(m)

    folium.LayerControl().add_to(m)

    display(m)

# インタラクティブに地図を更新する
interact(update_map, selected_date=date_selector, selected_geohash=geohash_selector)

In [ ]:
#個別でデータ作成している

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from ipywidgets import interact, Select, Dropdown
from IPython.display import display

def geohash_to_polygon(gh):
    #print(gh)
    if pd.isna(gh):
        return None
    
    bounds = geohash.bounds(gh)
    return Polygon([
        (bounds[0][1], bounds[0][0]),
        (bounds[0][1], bounds[1][0]),
        (bounds[1][1], bounds[1][0]),
        (bounds[1][1], bounds[0][0]),
        (bounds[0][1], bounds[0][0])
    ])


# データフレームを読み込み、必要な列をグループ化してジオハッシュの境界ボックスを計算
df = df_train.groupby(["取引時点", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()

# 各ジオハッシュレベルでのデータを個別にフィルタリング
df_geohash_2 = df.groupby(["取引時点", "geohash_2"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df_geohash_2['geometry'] = [geohash_to_polygon(geohash) for geohash in tqdm(df_geohash_2['geohash_2'],desc=f"Calculating geohash polygons level:2")]
gdf_geohash_2 = gpd.GeoDataFrame(df_geohash_2, geometry='geometry')
gdf_geohash_2.set_crs(epsg=4326, inplace=True)

df_geohash_3 = df.groupby(["取引時点", "geohash_3"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df_geohash_3['geometry'] = [geohash_to_polygon(geohash) for geohash in tqdm(df_geohash_3['geohash_3'],desc=f"Calculating geohash polygons level:3")]
gdf_geohash_3 = gpd.GeoDataFrame(df_geohash_3, geometry='geometry')
gdf_geohash_3.set_crs(epsg=4326, inplace=True)

df_geohash_4 = df.groupby(["取引時点", "geohash_4"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df_geohash_4['geometry'] = [geohash_to_polygon(geohash) for geohash in tqdm(df_geohash_4['geohash_4'],desc=f"Calculating geohash polygons level:4")]
gdf_geohash_4 = gpd.GeoDataFrame(df_geohash_4, geometry='geometry')
gdf_geohash_4.set_crs(epsg=4326, inplace=True)

df_geohash_5 = df.groupby(["取引時点", "geohash_5"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df_geohash_5['geometry'] = [geohash_to_polygon(geohash) for geohash in tqdm(df_geohash_5['geohash_5'],desc=f"Calculating geohash polygons level:5")]
gdf_geohash_5 = gpd.GeoDataFrame(df_geohash_5, geometry='geometry')
gdf_geohash_5.set_crs(epsg=4326, inplace=True)

df_geohash_6 = df.groupby(["取引時点", "geohash_6"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df_geohash_6['geometry'] = [geohash_to_polygon(geohash) for geohash in tqdm(df_geohash_6['geohash_6'],desc=f"Calculating geohash polygons level:6")]
gdf_geohash_6 = gpd.GeoDataFrame(df_geohash_6, geometry='geometry')
gdf_geohash_6.set_crs(epsg=4326, inplace=True)

df_geohash_7 = df.groupby(["取引時点", "geohash_7"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
df_geohash_7['geometry'] = [geohash_to_polygon(geohash) for geohash in tqdm(df_geohash_7['geohash_7'],desc=f"Calculating geohash polygons level:7")]
gdf_geohash_7 = gpd.GeoDataFrame(df_geohash_7, geometry='geometry')
gdf_geohash_7.set_crs(epsg=4326, inplace=True)

# 全てのジオハッシュレベルのデータフレームを連結
gdf_all = pd.concat([gdf_geohash_2, gdf_geohash_3, gdf_geohash_4, gdf_geohash_5, gdf_geohash_6, gdf_geohash_7])
unique_dates = gdf_all['取引時点'].unique()
unique_geohash_levels = ['geohash_2', 'geohash_3', 'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7']
date_selector = Select(options=unique_dates, description='取引時点:')
geohash_selector = Dropdown(options=unique_geohash_levels, description='Geohash Level:')

# 地図を更新する関数を定義
def update_map(selected_date, selected_geohash):
    m = folium.Map(location=[35.0, 135.0], zoom_start=5)
    if selected_geohash == 'geohash_2':
        selected_gdf = gdf_geohash_2[gdf_geohash_2['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_3':
        selected_gdf = gdf_geohash_3[gdf_geohash_3['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_4':
        selected_gdf = gdf_geohash_4[gdf_geohash_4['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_5':
        selected_gdf = gdf_geohash_5[gdf_geohash_5['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_6':
        selected_gdf = gdf_geohash_6[gdf_geohash_6['取引時点'] == selected_date]
    elif selected_geohash == 'geohash_7':
        selected_gdf = gdf_geohash_7[gdf_geohash_7['取引時点'] == selected_date]

    folium.Choropleth(
        geo_data=selected_gdf,
        name='choropleth',
        data=selected_gdf,
        columns=[selected_geohash, f"{TOTAL_PRICE_LOG}"],
        key_on=f'feature.properties.{selected_geohash}',
        fill_color='YlGnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='平均価格'
    ).add_to(m)

    folium.LayerControl().add_to(m)

    display(m)

interact(update_map, selected_date=date_selector, selected_geohash=geohash_selector)